## 0. DRIVE mount

In [1]:
# 구글 드라이브와 연동
from google.colab import drive
drive.mount("/content/gdrive")

# 연동 잘 되었는지 확인
!ls "gdrive/My Drive/Colab Notebooks/DACON/"


Mounted at /content/gdrive
k_fashion  LG_predict_complain


In [2]:
# zip 파일을 먼저 구글 클라우드에 업로드
# 압축파일을 세션 내 드라이브에 압축해제
!unzip "gdrive/My Drive/Colab Notebooks/DACON/LG_predict_complain/data.zip" -d 'data/'

Archive:  gdrive/My Drive/Colab Notebooks/DACON/LG_predict_complain/data.zip
  inflating: data/sample_submission.csv  
  inflating: data/test_err_data.csv  
  inflating: data/test_quality_data.csv  
  inflating: data/train_err_data.csv  
  inflating: data/train_problem_data.csv  
  inflating: data/train_quality_data.csv  


In [3]:
!pip install pycaret

     |████████████████████████████████| 256kB 8.5MB/s 
     |████████████████████████████████| 102kB 7.4MB/s 
     |████████████████████████████████| 6.8MB 12.5MB/s 
     |████████████████████████████████| 157.5MB 85kB/s 
     |████████████████████████████████| 1.8MB 31.7MB/s 
     |████████████████████████████████| 174kB 62.6MB/s 
     |████████████████████████████████| 1.6MB 46.7MB/s 
     |████████████████████████████████| 276kB 53.6MB/s 
     |████████████████████████████████| 14.2MB 349kB/s 
     |████████████████████████████████| 65.8MB 60kB/s 
     |████████████████████████████████| 245kB 48.1MB/s 
     |████████████████████████████████| 2.1MB 36.2MB/s 
     |████████████████████████████████| 348kB 52.6MB/s 
     |████████████████████████████████| 163kB 49.9MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 153kB 49.9MB/s 
     |████████████████████████████████| 1.1MB 33.8MB/s 

In [4]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

PATH = 'data/'

## 1. Model Train


In [5]:
"""
 학습에 사용되는 모든 컬럼
"""
all_data_col_err = ['err0', 'err1', 'err2', 'err3', 'err4', 'err5', 'err6', 'err7', 'err8', 'err9', 'err10', 
                    'err11', 'err12', 'err13', 'err14', 'err15', 'err16', 'err17', 'err18', 'err19', 'err20', 
                    'err21', 'err22', 'err23', 'err24', 'err25', 'err26', 'err27', 'err28', 'err29', 'err30', 
                    'err31', 'err32', 'err33', 'err34', 'err35', 'err36', 'err37', 'err38', 'err39', 'err40', 'err41']
# all_data_col_errcode = ['A','B','C','D','UNKNOWN']
all_data_col_errcode = ['A','B']
all_data_col_err_weekday = ['err_week_0', 'err_week_1', 'err_week_2', 'err_week_3', 'err_week_4', 'err_week_5', 'err_week_6']
all_data_col_err_hours = ['err_hours_0', 'err_hours_1', 'err_hours_2', 'err_hours_3', 'err_hours_4', 'err_hours_5', 'err_hours_6', 
                          'err_hours_7', 'err_hours_8', 'err_hours_9', 'err_hours_10', 'err_hours_11', 'err_hours_12', 'err_hours_13', 
                          'err_hours_14', 'err_hours_15', 'err_hours_16', 'err_hours_17', 'err_hours_18', 'err_hours_19', 'err_hours_20', 
                          'err_hours_21', 'err_hours_22', 'err_hours_23']

all_data_col_model = ['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'model_7', 'model_8']
all_data_col_fwver = ['03.11.1141', '03.11.1149', '03.11.1167', '04.16.2641', '04.16.3345', '04.16.3439', 
                      '04.16.3553', '04.16.3569', '04.16.3571', '04.22.1170', '04.22.1442', '04.22.1448', 
                      '04.22.1478', '04.22.1608', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', 
                      '04.22.1772', '04.22.1778', '04.33.1095', '04.33.1125', '04.33.1149', '04.33.1171', 
                      '04.33.1185', '04.33.1261', '04.73.2237', '04.73.2569', '04.73.2571', '04.73.2577', 
                      '04.82.1684', '04.82.1730', '04.82.1778', '05.15.2090', '05.15.2092', '05.15.2114', 
                      '05.15.2120', '05.15.2122', '05.15.2138', '05.15.3104', '05.66.3237', '05.66.3571', 
                      '10', '10.22.1770', '10.22.1780', '8.5.3']


all_data_col_qual = ['quality_0', 'quality_1', 'quality_2', 'quality_3', 'quality_4', 'quality_5', 'quality_6', 
                     'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
all_data_col_qual_weekday = ['qual_week_0', 'qual_week_1', 'qual_week_2', 'qual_week_3', 'qual_week_4', 'qual_week_5', 'qual_week_6']
all_data_col_qual_hours = ['qual_hours_0', 'qual_hours_1', 'qual_hours_2', 'qual_hours_3', 'qual_hours_4', 'qual_hours_5', 'qual_hours_6', 
                           'qual_hours_7', 'qual_hours_8', 'qual_hours_9', 'qual_hours_10', 'qual_hours_11', 'qual_hours_12', 'qual_hours_13', 
                           'qual_hours_14', 'qual_hours_15', 'qual_hours_16', 'qual_hours_17', 'qual_hours_18', 'qual_hours_19', 'qual_hours_20', 
                           'qual_hours_21', 'qual_hours_22', 'qual_hours_23']


all_data_columns = ['user_id'] + all_data_col_err + all_data_col_errcode + all_data_col_model + all_data_col_fwver + all_data_col_qual

### 1.1 Train data load

In [6]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_train_data_merge_2.csv')
df

,user_id,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,err39,err40,err41,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,03.11.1141,03.11.1149,03.11.1167,04.16.2641,04.16.3345,04.16.3439,04.16.3553,04.16.3569,04.16.3571,04.22.1170,04.22.1442,04.22.1448,04.22.1478,04.22.1608,04.22.1656,04.22.1666,04.22.1684,04.22.1750,04.22.1772,04.22.1778,04.33.1095,04.33.1125,04.33.1149,04.33.1171,04.33.1185,04.33.1261,04.73.2237,04.73.2569,04.73.2571,04.73.2577,04.82.1684,04.82.1730,04.82.1778,05.15.2090,05.15.2092,05.15.2114,05.15.2120,05.15.2122,05.15.2138,05.15.3104,05.66.3237,05.66.3571,10,10.22.1770,10.22.1780,8.5.3,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,quality_minus_0,quality_minus_1,quality_minus_2,quality_minus_3,quality_minus_4,quality_minus_5,quality_minus_6,quality_minus_7,quality_minus_8,quality_minus_9,quality_minus_10,quality_minus_11,quality_minus_12,err_week_0,err_week_1,err_week_2,err_week_3,err_week_4,err_week_5,err_week_6,qual_week_0,qual_week_1,qual_week_2,qual_week_3,qual_week_4,qual_week_5,qual_week_6,err_hours_0,err_hours_1,err_hours_2,err_hours_3,err_hours_4,err_hours_5,err_hours_6,err_hours_7,err_hours_8,err_hours_9,err_hours_10,err_hours_11,err_hours_12,err_hours_13,err_hours_14,err_hours_15,err_hours_16,err_hours_17,err_hours_18,err_hours_19,err_hours_20,err_hours_21,err_hours_22,err_hours_23,qual_hours_0,qual_hours_1,qual_hours_2,qual_hours_3,qual_hours_4,qual_hours_5,qual_hours_6,qual_hours_7,qual_hours_8,qual_hours_9,qual_hours_10,qual_hours_11,qual_hours_12,qual_hours_13,qual_hours_14,qual_hours_15,qual_hours_16,qual_hours_17,qual_hours_18,qual_hours_19,qual_hours_20,qual_hours_21,qual_hours_22,qual_hours_23,problem
0,10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,41.0,45.0,45.0,43.0,54.0,52.0,12.0,0.0,0.0,0.0,0.0,0.0,12.0,33.0,13.0,12.0,28.0,17.0,45.0,28.0,16.0,11.0,8.0,8.0,5.0,5.0,0.0,10.0,9.0,6.0,15.0,19.0,12.0,0.0,10.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
1,10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.0,0.0,2365.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,164.0,138.0,107.0,166.0,1534.0,117.0,139.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,34.0,68.0,27.0,21.0,0.0,2.0,22.0,29.0,57.0,28.0,83.0,160.0,169.0,183.0,193.0,210.0,153.0,85.0,230.0,176.0,258.0,70.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0.0,2.0,0.0,0.0,0.0,0.0,27.0,44.0,552.0,0.0,0.0,372.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,2.0

### 1.2 Data preprocess

In [9]:
np.log(df[all_data_col_err]).fillna(0)

,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,err39,err40,err41
0,-inf,-inf,2.079442,4.644391,-inf,0.000000,0.000000,-inf,-inf,1.94591,2.708050,2.772589,0.000000,2.302585,4.077537,4.110874,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,3.465736,0.000000,0.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
1,-inf,-inf,-inf,-inf,3.970292,0.000000,0.000000,-inf,-inf,-inf,2.302585,2.397895,0.000000,2.708050,5.017280,4.852030,-inf,1.386294,0.0,1.098612,0.0,6.628041,6.621406,1.609438,0.0,3.091042,-inf,-inf,-inf,-inf,5.521461,-inf,2.302585,2.890372,-inf,0.0,0.0,-inf,-inf,4.727388,4.025352,0.000000
2,-inf,-inf,0.693147,4.882802,0.000000,0.693147,0.000000,-inf,-inf,0.00000,2.564949,2.639057,0.000000,1.386294,3.951244,3.951244,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.693147,-inf,3.218876,0.693147,0.693147,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
3,-inf,-inf,-inf,-inf,0.693147,0.000000,0.000000,-inf,-inf,-inf,2.197225,2.197225,-inf,-inf,3.951244,3.401197,0.0,-inf,-inf,-inf,-inf,3.332205,2.944439,-inf,-inf,4.077537,-inf,-inf,-inf,-inf,4.174387,-inf,2.079442,-inf,-inf,0.0,0.0,0.693147,-inf,2.833213,0.000000,-inf
4,-inf,-inf,-inf,0.000000,-inf,1.098612,1.386294,-inf,-inf,-inf,2.772589,2.944439,1.098612,1.609438,4.962845,4.510860,-inf,-inf,-inf,-inf,-inf,4.941642,4.779123,-inf,-inf,3.496508,-inf,-inf,-inf,-inf,5.170484,-inf,2.772589,-inf,-inf,0.0,0.0,-inf,-inf,1.386294,-inf,0.693147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-inf,-inf,-inf,-inf,0.693147,1.609438,1.609438,-inf,-inf,-inf,1.609438,2.302585,2.079442,2.079442,3.258097,2.890372,-inf,-inf,-inf,-inf,-inf,2.833213,2.484907,-inf,-inf,3.091042,-inf,-inf,-inf,-inf,3.433987,-inf,1.609438,-inf,-inf,-inf,-inf,-inf,-inf,2.197225,1.945910,1.386294
14996,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.000000,0.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.000000,-inf,-inf,-inf,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
14997,-inf,-inf,-inf,0.000000,2.079442,0.000000,0.000000,-inf,-inf,-inf,2.772589,2.890372,1.386294,0.000000,5.198497,4.927254,-inf,-inf,-inf,-inf,-inf,3.912023,3.367296,-inf,-inf,3.891820,-inf,-inf,-inf,-inf,5.407172,-inf,2.772589,2.833213,-inf,0.0,0.0,-inf,-inf,4.060443,2.079442,1.609438
14998,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,2.708050,2.708050,-inf,0.000000,3.931826,2.484907,-inf,-inf,-inf,-inf,-inf,1.945910,0.000000,-inf,-inf,1.386294,-inf,-inf,-inf,-inf,2.639057,-inf,2.708050,2.484907,-inf,0.0,0.0,-inf,-inf,1.791759,-inf,-inf


In [ ]:
df.drop(['user_id'],axis=1,inplace=True)
df.drop(all_data_col_fwver,axis=1,inplace=True)
# df.drop(all_data_col_errcode,axis=1,inplace=True)
# df.drop(all_data_col_qual_weekday,axis=1,inplace=True)
# df.drop(all_data_col_qual_hours,axis=1,inplace=True)
drop_list = ['err28', 'quality_3', 'quality_4', 'quality_minus_3', 'quality_minus_4', 'quality_minus_7', 'quality_minus_8', 'quality_minus_9', 'quality_minus_10', 'quality_minus_12']
df.drop(drop_list, axis=1,inplace=True)
df.shape


(15000, 130)

In [ ]:
df['model_0']

0          0.0
1          0.0
2          0.0
3          0.0
4        777.0
         ...  
14995      0.0
14996      0.0
14997    826.0
14998    155.0
14999      0.0
Name: model_0, Length: 15000, dtype: float64

### 1.3 Model Train

In [ ]:
# AutoML
# data - 학습할 데이터가 있는 데이터 프레임, 
# target - 예측할 데이터 column
clf2 = setup(data = df, target = 'problem') 


,Description,Value
0,session_id,2938
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 130)"
5,Missing Values,False
6,Numeric Features,129
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
# create individual models for stacking
lgb_model = create_model('lightgbm')
catboost_model = create_model('catboost')
gbc_model = create_model('gbc')
et_model = create_model('et')
# stacking models
stacker = stack_models(estimator_list = [lgb_model,gbc_model,et_model], meta_model = catboost_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7952,0.8350,0.5217,0.7826,0.6261,0.4928,0.5120
1,0.7705,0.7944,0.4682,0.7397,0.5735,0.4271,0.4480
2,0.7800,0.7875,0.4566,0.7861,0.5777,0.4427,0.4726
3,0.7648,0.7924,0.4509,0.7324,0.5581,0.4100,0.4324
4,0.8086,0.8358,0.5347,0.8222,0.6480,0.5245,0.5474
5,0.7790,0.8030,0.4971,0.7478,0.5972,0.4534,0.4713
6,0.7924,0.8045,0.5145,0.7807,0.6202,0.4855,0.5055
7,0.8086,0.8444,0.5694,0.7912,0.6622,0.5335,0.5476
8,0.8133,0.8379,0.5520,0.8233,0.6609,0.5389,0.5594
9,0.7827,0.8154,0.5217,0.7407,0.6122,0.4675,0.4813


In [ ]:
# sort - metric 정렬 기준
# n_select - 상위 몇개 고를건지
best_5 = compare_models(sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7923,0.8151,0.5003,0.7974,0.6144,0.4824,0.5071,23.044
lightgbm,Light Gradient Boosting Machine,0.7879,0.8098,0.5095,0.7729,0.6138,0.4761,0.4959,0.833
gbc,Gradient Boosting Classifier,0.7866,0.8075,0.4715,0.8038,0.5939,0.4622,0.4927,6.612
et,Extra Trees Classifier,0.7792,0.8017,0.4675,0.7776,0.5834,0.4458,0.4727,2.184
rf,Random Forest Classifier,0.7737,0.7992,0.4445,0.7771,0.5652,0.4273,0.4579,3.110
xgboost,Extreme Gradient Boosting,0.7771,0.7944,0.5270,0.7270,0.6104,0.4598,0.4720,15.219
ada,Ada Boost Classifier,0.7735,0.7859,0.4718,0.7537,0.5800,0.4359,0.4585,1.365
lda,Linear Discriminant Analysis,0.7485,0.7362,0.3427,0.7717,0.4740,0.3396,0.3882,0.284
nb,Naive Bayes,0.6805,0.6689,0.1429,0.5711,0.2278,0.1105,0.1529,0.046
qda,Quadratic Discriminant Analysis,0.6991,0.6605,0.3784,0.5735,0.4543,0.2586,0.2702,0.179


In [ ]:
stacker = stack_models(estimator_list = best_5[1:], meta_model = best_5[0])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7752,0.8083,0.4684,0.7617,0.5801,0.4383,0.4624
1,0.8181,0.8574,0.5632,0.8340,0.6724,0.5529,0.5734
2,0.7790,0.8000,0.5086,0.7437,0.6041,0.4582,0.4742
3,0.7981,0.8141,0.5230,0.7982,0.6319,0.5010,0.5223
4,0.7686,0.8009,0.4971,0.7178,0.5874,0.4339,0.4480
5,0.8086,0.8258,0.5661,0.7976,0.6622,0.5339,0.5492
6,0.7867,0.8021,0.4741,0.8010,0.5957,0.4634,0.4928
7,0.7905,0.8005,0.4799,0.8107,0.6029,0.4730,0.5030
8,0.8038,0.8169,0.5072,0.8341,0.6308,0.5078,0.5369
9,0.7912,0.8229,0.5101,0.7832,0.6178,0.4829,0.5039


In [ ]:
# 모델 블렌딩
pred_holdout = predict_model(stacker)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.7934,0.8151,0.5131,0.8052,0.6268,0.4935,0.5174


In [ ]:
# 선정한 모델을 통한 soft 방식의 앙상블
blended = blend_models(estimator_list = best_5, fold = 3, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7949,0.8132,0.4858,0.8219,0.6106,0.4836,0.5144
1,0.7917,0.8153,0.4892,0.8054,0.6087,0.4781,0.5057
2,0.7811,0.8132,0.4763,0.7764,0.5904,0.4525,0.4776
Mean,0.7892,0.8139,0.4838,0.8012,0.6032,0.4714,0.4992
SD,0.0059,0.0010,0.0055,0.0188,0.0091,0.0136,0.0157


In [ ]:
# 모델 블렌딩
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7896,0.8142,0.4911,0.813,0.6124,0.4798,0.5085


In [ ]:
# 최종모델
final_model = finalize_model(stacker)

In [ ]:
save_model(stacker, '20210203_stacker_5bestmodel')

In [ ]:
stacker

StackingClassifier(cv=StratifiedKFold(n_splits=10, random_state=2938, shuffle=False),
                   estimators=[('lightgbm',
                                LGBMClassifier(boosting_type='gbdt',
                                               class_weight=None,
                                               colsample_bytree=1.0,
                                               importance_type='split',
                                               learning_rate=0.1, max_depth=-1,
                                               min_child_samples=20,
                                               min_child_weight=0.001,
                                               min_split_gain=0.0,
                                               n_estimators=100, n_jobs=-1,
                                               num_leaves=31, objective...
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
     

## 2. MODEL Predict

### 2.1 TEST Data Load

In [ ]:
test_df  = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_test_data_merge_2.csv')
test_df

,user_id,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,...,err_hours_8,err_hours_9,err_hours_10,err_hours_11,err_hours_12,err_hours_13,err_hours_14,err_hours_15,err_hours_16,err_hours_17,err_hours_18,err_hours_19,err_hours_20,err_hours_21,err_hours_22,err_hours_23,qual_hours_0,qual_hours_1,qual_hours_2,qual_hours_3,qual_hours_4,qual_hours_5,qual_hours_6,qual_hours_7,qual_hours_8,qual_hours_9,qual_hours_10,qual_hours_11,qual_hours_12,qual_hours_13,qual_hours_14,qual_hours_15,qual_hours_16,qual_hours_17,qual_hours_18,qual_hours_19,qual_hours_20,qual_hours_21,qual_hours_22,qual_hours_23
0,30000,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,...,12.0,123.0,30.0,11.0,32.0,31.0,105.0,284.0,157.0,133.0,143.0,522.0,475.0,97.0,113.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0
1,30001,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.0,5.0,20.0,12.0,25.0,1.0,9.0,5.0,4.0,14.0,9.0,24.0,12.0,8.0,9.0,19.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
2,30002,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,...,37.0,26.0,31.0,55.0,26.0,34.0,32.0,47.0,58.0,63.0,92.0,38.0,10.0,23.0,14.0,16.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,24.0,0.0,36.0,0.0,12.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
3,30003,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,...,14.0,13.0,6.0,4.0,23.0,14.0,6.0,11.0,29.0,13.0,10.0,31.0,13.0,17.0,20.0,10.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0
4,30004,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,...,99.0,43.0,11.0,11.0,5.0,37.0,66.0,55.0,34.0,32.0,31.0,27.0,32.0,24.0,13.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,179.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,13.0,34.0,78.0,68.0,36.0,101.0,37.0,89.0,72.0,97.0,46.0,62.0,128.0,88.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14995,44995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,183.0,0.0,14.0,1.0,0.0,1.0,1.0,0.0,0.0,...,3.0,12.0,27.0,35.0,46.0,24.0,20.0,12.0,14.0,14.0,29.0,27.0,59.0,18.0,32.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,44996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,0.0,0.0,0.0,138.0,165.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,163.0,0.0,12.0,1118.0,0.0,1.0,1.0,1.0,0.0,...,21.0,28.0,174.0,56.0,110.0,27.0,29.0,113.0,183.0,103.0,103.0,273.0,129.0,164.0,189.0,50.0,12.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,24.0
14997,44997,0.0,0.0

### 2.2 DATA preprocess

In [ ]:
test_df.drop(['user_id'],axis=1,inplace=True)
test_df.drop(all_data_col_fwver,axis=1,inplace=True)
# test_df.drop(all_data_col_errcode,axis=1,inplace=True)

drop_list = ['err28', 'quality_3', 'quality_4', 'quality_minus_3', 'quality_minus_4', 'quality_minus_7', 'quality_minus_8', 'quality_minus_9', 'quality_minus_10', 'quality_minus_12']
test_df.drop(drop_list, axis=1,inplace=True)
test_df.shape

(14999, 129)

In [ ]:
print(list(test_df.columns))

['err0', 'err1', 'err2', 'err3', 'err4', 'err5', 'err6', 'err7', 'err8', 'err9', 'err10', 'err11', 'err12', 'err13', 'err14', 'err15', 'err16', 'err17', 'err18', 'err19', 'err20', 'err21', 'err22', 'err23', 'err24', 'err25', 'err26', 'err27', 'err28', 'err29', 'err30', 'err31', 'err32', 'err33', 'err34', 'err35', 'err36', 'err37', 'err38', 'err39', 'err40', 'err41', 'model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'model_7', 'model_8', 'quality_0', 'quality_1', 'quality_2', 'quality_3', 'quality_4', 'quality_5', 'quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'quality_minus_0', 'quality_minus_1', 'quality_minus_2', 'quality_minus_3', 'quality_minus_4', 'quality_minus_5', 'quality_minus_6', 'quality_minus_7', 'quality_minus_8', 'quality_minus_9', 'quality_minus_10', 'quality_minus_11', 'quality_minus_12', 'err_week_0', 'err_week_1', 'err_week_2', 'err_week_3', 'err_week_4', 'err_week_5', 'err_week_6', 'qual_week_0

### 2.3 TEST data predict

In [ ]:
today = '20210204'
# autoML Predict
predictions = predict_model(stacker, data = test_df)

x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i] =='1.0':
    x.append(predictions['Score'][i])
  else:
    x.append(1-predictions['Score'][i])

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv(f"{today}_KMH_automl_5select_3fold_stack_col130_0.8151.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.9526
1,30001,0.1748
2,30002,0.2884
3,30003,0.7836
4,30004,0.8111
...,...,...
14994,44994,0.3913
14995,44995,0.2914
14996,44996,0.3732
14997,44997,0.9408


In [ ]:
!cp -rf 'logs.log' '/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/history/20210129_automl_model_train_LB0.811_PB0.8242.log'

In [ ]:
!cp -rf 'catboost_info' '/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/history/20210129_automl_model_train_LB0.811_PB0.8242_catboost_info'